In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchvision import datasets, transforms
# from PIL import Image

In [2]:
BATCH_SIZE=32 #大概需要2G的显存
EPOCHS=5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class train(Dataset):
    def __init__(self):
        
        mnist_train = pd.read_csv('mnist_train.csv')
        train = mnist_train.values

        index = np.where(train[:, 0] == 0)
        for i in index:
            mnist_train_0 = train[i]
        # train number 1
        index = np.where(train[:, 0] == 1)
        for i in index:
            mnist_train_1 = train[i]
            
        nrow = 28
        ncol = 28

        # X and label
        x1 = mnist_train_1[:, 1:]
        x0 = mnist_train_0[:, 1:]
        y1 = mnist_train_1[:, 0]
        y0 = mnist_train_0[:, 0]

        # X reshape to 28 28
        nx1 = x1.shape[0]
        nx0 = x0.shape[0]
        x1_reshape = x1.reshape((nx1, nrow, ncol))
        x0_reshape = x0.reshape((nx0, nrow, ncol))

        def left_top(x):
            nsam = x.shape[0]
            nrow = x.shape[1]
            ncol = x.shape[2]
            z = np.zeros((nsam, nrow, ncol), dtype=np.double)
            data1 = np.concatenate((x, z), axis=1)
            data2 = np.concatenate((z, z), axis=1)
            data3 = np.concatenate((data1, data2), 2)

            return data3

        def right_bottom(x):
            nsam = x.shape[0]
            nrow = x.shape[1]
            ncol = x.shape[2]
            z = np.zeros((nsam, nrow, ncol), dtype=np.double)
            data1 = np.concatenate((z, z), axis=1)
            data2 = np.concatenate((z, x), axis=1)
            data3 = np.concatenate((data1, data2), 2)
            return data3

        left0 = left_top(x0_reshape)
        right1 = right_bottom(x1_reshape)
        train_data = np.concatenate((left0, right1), axis=0)
        nn = train_data.shape[0]

        # reshape to 3136 which is 56*56
        train_data_reshape = train_data
        train_data_reshape = torch.from_numpy(train_data_reshape)
        train_data_reshape = torch.unsqueeze(train_data_reshape, 1)
        yy = np.concatenate((y0, y1), axis=0)
        
        yy=torch.from_numpy(yy)
        #         yy= y[:,np.newaxis];

        #         xy=np.concatenate((yy,train_data_reshape),axis=1)

        self.len = yy.shape[0]

        self.x_data = train_data_reshape

        self.y_data = yy
        # self.transform = transform

    def __getitem__(self, index):
#         if self.transform:
#             x = self.transform(x)
        #img = Image.fromarray(self.x_data[index])
        return self.x_data[index], self.y_data[index]
      

    def __len__(self):
        return self.len
dataset1= train()
train_loader = DataLoader(dataset=dataset1,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [4]:
class test(Dataset):
    def __init__(self):
  
        mnist_test = pd.read_csv('mnist_test.csv')
        testtest= mnist_test.values
        index = np.where(testtest[:, 0] == 0)
        for i in index:
            test0 = testtest[i]
        # train number 1
        index = np.where(testtest[:, 0] == 1)
        for i in index:
            test1 = testtest[i]
       
        # train0and1=np.concatenate((mnist_train_0, mnist_train_1), axis=0)
        nrow = 28
        ncol = 28

        # X and label
        x1 = test1[:, 1:]
        x0 = test0[:, 1:]
        y1 = test1[:, 0]
        y0 = test0[:, 0]

        # X reshape to 28 28
        nx1 = x1.shape[0]
        nx0 = x0.shape[0]
        x1_reshape = x1.reshape((nx1, nrow, ncol))
        x0_reshape = x0.reshape((nx0, nrow, ncol))

        def left_top(x):
            nsam = x.shape[0]
            nrow = x.shape[1]
            ncol = x.shape[2]
            z = np.zeros((nsam, nrow, ncol), dtype=np.double)
            data1 = np.concatenate((x, z), axis=1)
            data2 = np.concatenate((z, z), axis=1)
            data3 = np.concatenate((data1, data2), 2)

            return data3

        def right_bottom(x):
            nsam = x.shape[0]
            nrow = x.shape[1]
            ncol = x.shape[2]
            z = np.zeros((nsam, nrow, ncol), dtype=np.double)
            data1 = np.concatenate((z, z), axis=1)
            data2 = np.concatenate((z, x), axis=1)
            data3 = np.concatenate((data1, data2), 2)
            return data3

        left1 = right_bottom(x1_reshape)
        right0 = left_top(x0_reshape)
        train_data = np.concatenate((left1, right0), axis=0)
#         train_data=left1
        nn = train_data.shape[0]

        # reshape to 3136 which is 56*56
        train_data_reshape = train_data
        train_data_reshape = torch.from_numpy(train_data_reshape)
        train_data_reshape = torch.unsqueeze(train_data_reshape, 1)
        yy = np.concatenate((y1, y0), axis=0)
#         yy=y1

        yy=torch.from_numpy(yy)
        self.len = yy.shape[0]

        self.x_data = train_data_reshape

        self.y_data = yy
        
        # self.transform = transform

    def __getitem__(self, index):
        # img = Image.fromarray(self.x_data[index])
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
dataset2 = test()
test_loader = DataLoader(dataset=dataset2,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 1,28x28
        self.conv1=nn.Conv2d(1,10,5) # 10, 24x24
        self.conv2=nn.Conv2d(10,20,3) # 128, 10x10
        self.fc1 = nn.Linear(2880,300)
        self.fc2 = nn.Linear(300,2)
    def forward(self,x):
        in_size = x.size(0)
        out = self.conv1(x) #24
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = self.conv2(out) #10
        out = F.relu(out)
        out = F.max_pool2d(out, 2, 2)  #12
        out = F.relu(out)   
        out = out.view(in_size,-1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        # out = F.log_softmax(out,dim=1)
        # out = F.softmax(out,dim=1)
        return out

In [6]:
model = ConvNet()
model = model.to(DEVICE)
model= model.double()
optimizer = optim.Adam(model.parameters())
# criterion = nn.CrossEntropyLoss()

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        # print(data.size(), target.size())
        optimizer.zero_grad()
        output=model(data)
        # print(output)
        #print(target)
        # loss = F.nll_loss(output, target)
        # print(loss)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.double())
#             print(output)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.4f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))





In [8]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [1568/12665 (12%)]	Loss: 0.000000
Train Epoch: 1 [3168/12665 (25%)]	Loss: 0.000000
Train Epoch: 1 [4768/12665 (38%)]	Loss: 0.000000
Train Epoch: 1 [6368/12665 (50%)]	Loss: 0.000000
Train Epoch: 1 [7968/12665 (63%)]	Loss: 0.000000
Train Epoch: 1 [9568/12665 (76%)]	Loss: 0.000000
Train Epoch: 1 [11168/12665 (88%)]	Loss: 0.000000

Test set: Average loss: 0.0000, Accuracy: 2115/2115 (100.0000%)

Train Epoch: 2 [1568/12665 (12%)]	Loss: 0.000000
Train Epoch: 2 [3168/12665 (25%)]	Loss: 0.000000
Train Epoch: 2 [4768/12665 (38%)]	Loss: 0.000000
Train Epoch: 2 [6368/12665 (50%)]	Loss: 0.000000
Train Epoch: 2 [7968/12665 (63%)]	Loss: 0.000000
Train Epoch: 2 [9568/12665 (76%)]	Loss: 0.000000
Train Epoch: 2 [11168/12665 (88%)]	Loss: 0.000000

Test set: Average loss: 0.0000, Accuracy: 2115/2115 (100.0000%)

Train Epoch: 3 [1568/12665 (12%)]	Loss: 0.000000
Train Epoch: 3 [3168/12665 (25%)]	Loss: 0.000000
Train Epoch: 3 [4768/12665 (38%)]	Loss: 0.000000
Train Epoch: 3 [6368/12665 (50%)]